In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

### 파일 불러오기 

In [3]:
# 1. 파일 불러오기
fid_data_24 = pd.read_csv('2024_수비_statiz.csv')
bat_data_24 = pd.read_csv('2024_타격_statiz.csv')
pit_data_24 = pd.read_csv('2024_투구_statiz.csv')
sal_data_24 = pd.read_csv('2024_연봉_statiz.csv')

### 필요없는 칼럼 제거

In [5]:
fid_data_24 = fid_data_24.drop(index=0).reset_index(drop=True)
bat_data_24 = bat_data_24.drop(index=0).reset_index(drop=True)
pit_data_24 = pit_data_24.drop(index=0).reset_index(drop=True)

fid_data_24 = fid_data_24.drop(columns=['Rank', 'Sort▼'])
bat_data_24 = bat_data_24.drop(columns=['Rank', 'Sort▼'])
pit_data_24 = pit_data_24.drop(columns=['Rank', 'Sort▼'])

### 데이터 칼럼 맞추기

In [7]:
# 각 데이터프레임의 칼럼명 집합
fid_cols = set(fid_data_24.columns)
pit_cols = set(pit_data_24.columns)

# 공통(중복) 칼럼명 집합
common_cols = fid_cols & pit_cols

# 결과 출력
print(f"중복(공통) 칼럼 개수: {len(common_cols)}")
print(f"중복(공통) 칼럼명: {list(common_cols)}")

중복(공통) 칼럼 개수: 7
중복(공통) 칼럼명: ['GS', 'IP', 'pid', 'G', 'Name', 'Team', '팀']


In [8]:
# 중복되는 칼럼명 리스트
cols_to_rename = ['G', 'IP', 'GS']

# fid_data_24: fid_ 붙이기
fid_rename_dict = {col: f'fid_{col}' for col in cols_to_rename if col in fid_data_24.columns}
fid_data_24 = fid_data_24.rename(columns=fid_rename_dict)

# pit_data_24: pit_ 붙이기
pit_rename_dict = {col: f'pit_{col}' for col in cols_to_rename if col in pit_data_24.columns}
pit_data_24 = pit_data_24.rename(columns=pit_rename_dict)

In [22]:
# Team 칼럼명 변경 -> 포지션
bat_data_24 = bat_data_24.rename(columns={'Team': '포지션'})
bat_data_24['포지션'] = bat_data_24['포지션'].str.replace(r'\s+', ' ', regex=True)  # 공백 통일
bat_data_24['포지션'] = bat_data_24['포지션'].str.replace(r'^\d+\s*', '', regex=True)  # 맨 앞 숫자만 제거

fid_data_24 = fid_data_24.rename(columns={'Team': '포지션'})
fid_data_24['포지션'] = fid_data_24['포지션'].str.replace(r'\s+', ' ', regex=True)  # 공백 통일
fid_data_24['포지션'] = fid_data_24['포지션'].str.replace(r'^\d+\s*', '', regex=True)  # 맨 앞 숫자만 제거

pit_data_24 = pit_data_24.rename(columns={'Team': '포지션'})
pit_data_24['포지션'] = pit_data_24['포지션'].str.replace(r'\s+', ' ', regex=True)  # 공백 통일
pit_data_24['포지션'] = pit_data_24['포지션'].str.replace(r'^\d+\s*', '', regex=True)  # 맨 앞 숫자만 제거

pos_map = {
    'C': '포수',
    '1B': '1루수',
    '2B': '2루수',
    '3B': '3루수',
    'SS': '유격수',
    'LF': '좌익수',
    'CF': '중견수',
    'RF': '우익수',
    'DH': '지명타자', 
    'P': '투수'
}


bat_data_24['포지션'] = bat_data_24['포지션'].map(pos_map).fillna(bat_data_24['포지션'])
fid_data_24['포지션'] = fid_data_24['포지션'].map(pos_map).fillna(fid_data_24['포지션'])
pit_data_24['포지션'] = pit_data_24['포지션'].map(pos_map).fillna(pit_data_24['포지션'])

#display(bat_data_24)
#display(fid_data_24)
#display(pit_data_24)

,Name,포지션,pit_G,pit_GS,GR,GF,CG,SHO,W,L,...,WP,ERA,RA9,rRA9,rRA9pf,FIP,WHIP,WAR▼,팀,pid
0,류현진,투수,28,28,0,0,0,0,10,8,...,1,3.87,4.43,4.37,4.41,3.67,1.36,4.44,한화 이글스,10590.0
1,주현상,투수,65,0,65,51,0,0,8,4,...,1,2.65,2.78,3.12,3.09,3.85,0.84,3.13,한화 이글스,11415.0
2,와이스,투수,16,16,0,0,0,0,5,5,...,6,3.73,4.22,4.12,4.06,3.65,1.16,2.92,한화 이글스,16153.0
3,문동주,투수,21,21,0,0,0,0,7,7,...,8,5.17,5.74,5.63,5.61,4.71,1.67,1.77,한화 이글스,15013.0
4,바리아,투수,20,19,1,0,0,0,6,7,...,6,5.15,5.63,5.5,5.39,3.85,1.42,1.69,한화 이글스,16150.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,전준표,투수,18,5,13,4,0,0,2,4,...,4,6.83,9.11,8.78,8.93,7.09,1.99,-0.35,키움 히어로즈,16122.0
289,오석주,투수,17,0,17,7,0,0,1,1,...,2,11.12,11.12,10.67,11.22,4.4,2.29,-0.5,키움 히어로즈,12860.0
290,조영건,투수,25,3,22,3,0,0,2,1,...,3,8.01,8.92,8.13,8.83,6.01,1.91,-0.55,키움 히어로즈,14132.0
291,윤석원,투수,11,1,10,3,0,0,0,1,...,3,11.42,12.46,12.57,12.91,7.72,2.19,-0.66,키움 히어로즈,15071.0


### 데이터 병합하기

In [12]:
all_cols = ['포지션', 'G', 'IP', 'pid', 'GS', '팀', 'Name']
cols_to_rename = ['G', 'IP', 'GS']

other_cols = [col for col in all_cols if col not in cols_to_rename]
print(other_cols)

['포지션', 'pid', '팀', 'Name']


In [13]:
# 병합(merge)할 때 기준 키로 사용
merged_base_1st = pd.merge(
    fid_data_24,
    pit_data_24,
    how='outer',  # 또는 'inner' 등 필요에 따라
    on=['포지션', 'pid', '팀', 'Name'],
    suffixes=('_fid', '_pit')  # 혹시 겹치는 칼럼이 있으면 접미사로 구분
)

In [14]:
merged_base_1st

,Name,포지션,fid_G,fid_GS,fid_IP,TC,PO,Ass,E,F%,...,ROE,BK,WP,ERA,RA9,rRA9,rRA9pf,FIP,WHIP,WAR▼
0,김선빈,1루수,1,0,1.0,2,2,0,0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,안치홍,1루수,37,34,293.0,268,240,24,4,98.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,최정,1루수,1,0,1.0,0,0,0,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,이원석,1루수,12,12,87.0,91,85,4,2,97.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,최주환,1루수,122,116,1027.2,972,906,60,6,99.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
905,윤준호,포수,3,0,9.0,5,5,0,0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
906,박민준,포수,4,1,10.0,10,10,0,0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
907,서동욱,포수,32,5,95.0,86,75,10,1,98.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
908,류현준,포수,1,0,3.0,4,3,1,0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# 각 데이터프레임의 칼럼명 집합
merged_cols = set(merged_base_1st.columns)
bat_cols = set(bat_data_24.columns)

# 공통(중복) 칼럼명 집합
common_cols_2nd = merged_cols & bat_cols

# 결과 출력
print(f"중복(공통) 칼럼 개수: {len(common_cols_2nd)}")
print(f"중복(공통) 칼럼명: {list(common_cols_2nd)}")

중복(공통) 칼럼 개수: 14
중복(공통) 칼럼명: ['WAR▼', '2B', 'HP', 'BB', 'SO', 'HR', 'pid', '3B', 'Name', 'H', '포지션', 'IB', '팀', 'R']


In [24]:
# 중복(공통) 칼럼 리스트 
common_cols = ['R', 'H', 'HP', 'WAR▼', 'HR', '3B', 'SO', 'BB', '2B', 'IB'] 

# bat_data_24에서 실제로 존재하는 공통 칼럼만 추출
bat_present_cols = [col for col in common_cols if col in bat_data_24.columns]

# prefix 붙여서 이름 변경 dict 생성
bat_renamed = {col: f"bat_{col}" for col in bat_present_cols}
bat_data_24 = bat_data_24.rename(columns=bat_renamed)

bat_data_24

,Name,포지션,G,oWAR,dWAR,PA,ePA,AB,bat_R,bat_H,...,SF,비율,비율.1,비율.2,비율.3,비율.4,비율.5,bat_WAR▼,팀,pid
0,노시환,루수,136,2.5,-0.08,601,597,526,88,143,...,4,0.272,0.356,0.454,0.81,0.145,106.5,2.41,한화 이글스,14125.0
1,페라자,좌익수,122,2.27,-0.14,522,518,455,75,125,...,2,0.275,0.364,0.486,0.85,0.158,116.7,2.13,한화 이글스,16121.0
2,최재훈,포수,116,1.69,0.13,348,340,280,34,72,...,6,0.257,0.371,0.336,0.707,0.131,96.6,1.82,한화 이글스,10170.0
3,황영묵,루수,123,1.38,0.01,389,386,349,52,105,...,1,0.301,0.365,0.372,0.737,0.129,95.1,1.39,한화 이글스,16110.0
4,안치홍,지명타자,128,1.65,-0.31,533,531,473,64,142,...,4,0.3,0.37,0.427,0.797,0.146,107.9,1.33,한화 이글스,10014.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,예진원,중견수,22,-0.1,-0.39,21,21,19,2,4,...,0,0.211,0.286,0.263,0.549,0.056,42.8,-0.5,키움 히어로즈,13093.0
386,이원석,지명타자,39,-0.33,-0.2,109,109,91,5,20,...,2,0.22,0.33,0.264,0.594,0.085,66.4,-0.53,키움 히어로즈,10180.0
387,이승원,유격수,20,-0.53,-0.2,41,39,37,3,4,...,0,0.108,0.154,0.135,0.289,-0.058,-43.9,-0.74,키움 히어로즈,15476.0
388,임지열,우익수,22,-0.59,-0.28,57,57,49,4,5,...,1,0.102,0.211,0.163,0.374,-0.011,-8.3,-0.87,키움 히어로즈,11225.0


In [26]:
# 병합(merge)할 때 기준 키로 사용
merged_base_2nd = pd.merge(
    merged_base_1st,
    bat_data_24,
    how='outer',  # 또는 'inner' 등 필요에 따라
    on=['포지션', 'pid', '팀', 'Name'],
)
#merged_base_2nd

In [ ]:
# 705 390 293 855

In [28]:
sal_data_24 = sal_data_24.rename(columns={'선수':'Name'})

display(merged_base_2nd)
display(sal_data_24)

,Name,포지션,fid_G,fid_GS,fid_IP,TC,PO,Ass,E,F%,...,GDP,SH,SF,비율,비율.1,비율.2,비율.3,비율.4,비율.5,bat_WAR▼
0,김선빈,1루수,1,0,1.0,2,2,0,0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,안치홍,1루수,37,34,293.0,268,240,24,4,98.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,최정,1루수,1,0,1.0,0,0,0,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,이원석,1루수,12,12,87.0,91,85,4,2,97.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,최주환,1루수,122,116,1027.2,972,906,60,6,99.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029,박민준,포수,4,1,10.0,10,10,0,0,100.0,...,0,0,0,0.0,0.0,0.0,0.0,-0.19,-144.6,-0.14
1030,서동욱,포수,32,5,95.0,86,75,10,1,98.8,...,1,0,0,0.115,0.148,0.154,0.302,-0.052,-37.8,-0.45
1031,류현준,포수,1,0,3.0,4,3,1,0,100.0,...,0,0,0,0.667,0.667,1.0,1.667,0.487,380.3,0.12
1032,백두산,포수,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0.0,0.0,0.0,0.0,-0.19,-127.9,-0.02


,Name,연봉(만원),WAR,WAR당 연봉,팀,pid
0,류현진,250000,4.56,54775,한화 이글스,10590
1,채은성,100000,1.40,71340,한화 이글스,11215
2,페냐,65000,0.35,187034,한화 이글스,15146
3,페라자,60000,2.90,20686,한화 이글스,16121
4,최재훈,60000,1.68,35760,한화 이글스,10170
...,...,...,...,...,...,...
850,박준형,3000,-0.25,-12108,키움 히어로즈,14211
851,박윤성,3000,0.04,72014,키움 히어로즈,15475
852,서유신,3000,-,-,키움 히어로즈,15483
853,이명기,3000,-,-,키움 히어로즈,14212


In [30]:
baseball_player_data = pd.merge(
    merged_base_2nd,
    sal_data_24,
    how='left',
    on=['pid', '팀'],  # 동명이인 방지
)

In [32]:
baseball_player_data

,Name_x,포지션,fid_G,fid_GS,fid_IP,TC,PO,Ass,E,F%,...,비율.1,비율.2,비율.3,비율.4,비율.5,bat_WAR▼,Name_y,연봉(만원),WAR,WAR당 연봉
0,김선빈,1루수,1,0,1.0,2,2,0,0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,김선빈,60000,2.11,28379
1,안치홍,1루수,37,34,293.0,268,240,24,4,98.5,...,NaN,NaN,NaN,NaN,NaN,NaN,안치홍,50000,1.97,25319
2,최정,1루수,1,0,1.0,0,0,0,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,최정,100000,4.68,21383
3,이원석,1루수,12,12,87.0,91,85,4,2,97.8,...,NaN,NaN,NaN,NaN,NaN,NaN,이원석,40000,-0.47,-84662
4,최주환,1루수,122,116,1027.2,972,906,60,6,99.4,...,NaN,NaN,NaN,NaN,NaN,NaN,최주환,65000,0.35,184816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029,박민준,포수,4,1,10.0,10,10,0,0,100.0,...,0.0,0.0,0.0,-0.19,-144.6,-0.14,박민준,3000,-0.13,-22413
1030,서동욱,포수,32,5,95.0,86,75,10,1,98.8,...,0.148,0.154,0.302,-0.052,-37.8,-0.45,서동욱,3300,-0.48,-6922
1031,류현준,포수,1,0,3.0,4,3,1,0,100.0,...,0.667,1.0,1.667,0.487,380.3,0.12,류현준,3000,0.15,19631
1032,백두산,포수,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,-0.19,-127.9,-0.02,백두산,3000,-0.02,-150583


In [34]:
baseball_player_data = baseball_player_data.fillna(0)
baseball_player_data

,Name_x,포지션,fid_G,fid_GS,fid_IP,TC,PO,Ass,E,F%,...,비율.1,비율.2,비율.3,비율.4,비율.5,bat_WAR▼,Name_y,연봉(만원),WAR,WAR당 연봉
0,김선빈,1루수,1,0,1.0,2,2,0,0,100.0,...,0,0,0,0,0,0,김선빈,60000,2.11,28379
1,안치홍,1루수,37,34,293.0,268,240,24,4,98.5,...,0,0,0,0,0,0,안치홍,50000,1.97,25319
2,최정,1루수,1,0,1.0,0,0,0,0,0.0,...,0,0,0,0,0,0,최정,100000,4.68,21383
3,이원석,1루수,12,12,87.0,91,85,4,2,97.8,...,0,0,0,0,0,0,이원석,40000,-0.47,-84662
4,최주환,1루수,122,116,1027.2,972,906,60,6,99.4,...,0,0,0,0,0,0,최주환,65000,0.35,184816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029,박민준,포수,4,1,10.0,10,10,0,0,100.0,...,0.0,0.0,0.0,-0.19,-144.6,-0.14,박민준,3000,-0.13,-22413
1030,서동욱,포수,32,5,95.0,86,75,10,1,98.8,...,0.148,0.154,0.302,-0.052,-37.8,-0.45,서동욱,3300,-0.48,-6922
1031,류현준,포수,1,0,3.0,4,3,1,0,100.0,...,0.667,1.0,1.667,0.487,380.3,0.12,류현준,3000,0.15,19631
1032,백두산,포수,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,-0.19,-127.9,-0.02,백두산,3000,-0.02,-150583


In [36]:
baseball_player_data.columns

Index(['Name_x', '포지션', 'fid_G', 'fid_GS', 'fid_IP', 'TC', 'PO', 'Ass', 'E',
       'F%', 'RF9', '수비 관련 득점 기여도', '수비 관련 득점 기여도.1', '수비 관련 득점 기여도.2',
       '수비 관련 득점 기여도.3', '수비 관련 득점 기여도.4', '수비 관련 득점 기여도.5', '수비 관련 득점 기여도.6',
       '수비 관련 득점 기여도.7', '수비 관련 득점 기여도.8', '수비 관련 득점 기여도.9', 'POSAdj',
       'RAAwithPOS', 'WAAwoPOS', 'WAAwithPOS▼', '팀', 'pid', 'pit_G', 'pit_GS',
       'GR', 'GF', 'CG', 'SHO', 'W', 'L', 'S', 'HD', 'pit_IP', 'ER', 'R',
       'rRA', 'TBF', 'H', '2B', '3B', 'HR', 'BB', 'HP', 'IB', 'SO', 'ROE',
       'BK', 'WP', 'ERA', 'RA9', 'rRA9', 'rRA9pf', 'FIP', 'WHIP', 'WAR▼', 'G',
       'oWAR', 'dWAR', 'PA', 'ePA', 'AB', 'bat_R', 'bat_H', 'bat_2B', 'bat_3B',
       'bat_HR', 'TB', 'RBI', 'SB', 'CS', 'bat_BB', 'bat_HP', 'bat_IB',
       'bat_SO', 'GDP', 'SH', 'SF', '비율', '비율.1', '비율.2', '비율.3', '비율.4',
       '비율.5', 'bat_WAR▼', 'Name_y', '연봉(만원)', 'WAR', 'WAR당 연봉'],
      dtype='object')

In [ ]:
# baseball_player_data.to_csv('baseball_player_data.csv', index=False, encoding='utf-8-sig')